# 🤖 Multi-LLM Optimizer - POC Complet (Option A)

**Objectif**: Démonstration du système multi-LLM pour optimisation automatique de stratégies trading.

**Architecture**:
- **Analyst Agent** (deepseek-r1:70b): Analyse quantitative des résultats de sweep
- **Strategist Agent** (gpt-oss:20b): Génération créative de propositions de paramètres
- **BacktestEngine** (GPU): Tests automatiques des propositions

**Workflow**:
1. Sweep initial (grid search GPU)
2. Analyse LLM-A → Identification patterns
3. Propositions LLM-B → 3 configs créatives
4. Tests automatiques → Validation performances
5. Visualisation → Comparaison baseline vs LLM

**Prérequis**:
- Ollama running (`ollama serve`)
- Modèles téléchargés: `deepseek-r1:70b`, `gpt-oss:20b`
- GPU disponible (RTX 5090/2060)
- Données historiques BTCUSDT chargées

In [ ]:
# Imports système
import sys
import warnings
from pathlib import Path

# Ajouter src/ au PYTHONPATH
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Imports scientifiques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Imports ThreadX
from threadx.backtest.engine import BacktestEngine
from threadx.ui.strategy_registry import get_strategy_class
from threadx.llm.agents.analyst import Analyst
from threadx.llm.agents.strategist import Strategist
from threadx.llm.client import LLMClient

# Configuration
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports réussis")
print(f"📁 Project root: {project_root}")
print(f"🐍 Python: {sys.version.split()[0]}")

## 🎯 Section 2: Définition des paramètres du POC

Configuration de la stratégie MA_Crossover et paramètres de sweep.

In [ ]:
# Configuration stratégie MA_Crossover
STRATEGY_NAME = "MA_Crossover"

# Paramètres de base (baseline)
BASELINE_PARAMS = {
    "short_period": 10,
    "long_period": 30,
    "use_ema": False,
}

# Specs des paramètres (contraintes pour Strategist)
PARAM_SPECS = {
    "short_period": {"min": 5, "max": 50, "step": 5, "type": int},
    "long_period": {"min": 20, "max": 100, "step": 10, "type": int},
    "use_ema": {"type": bool},
}

# Configuration sweep initial
SWEEP_CONFIG = {
    "short_period": [5, 10, 15, 20],  # 4 valeurs
    "long_period": [30, 50, 70],       # 3 valeurs
    "use_ema": [False, True],          # 2 valeurs
}
# Total: 4 * 3 * 2 = 24 configurations

# Période de données
DATA_START = "2024-01-01"
DATA_END = "2024-12-31"
SYMBOL = "BTCUSDT"
TIMEFRAME = "1h"

print("✅ Paramètres définis:")
print(f"   Strategy: {STRATEGY_NAME}")
print(f"   Baseline: {BASELINE_PARAMS}")
print(f"   Sweep configs: {np.prod([len(v) for v in SWEEP_CONFIG.values()])}")
print(f"   Data: {SYMBOL} {TIMEFRAME} [{DATA_START} → {DATA_END}]")

In [ ]:
# Charger données depuis cache/CSV (simulé pour POC)
# Dans production: data_loader.load_historical(SYMBOL, TIMEFRAME, DATA_START, DATA_END)

# Générer données synthétiques pour démo rapide
np.random.seed(42)
n_candles = 8760  # ~1 an de données horaires

dates = pd.date_range(DATA_START, periods=n_candles, freq='1h')
close_prices = 40000 + np.cumsum(np.random.randn(n_candles) * 100)  # Random walk

df_market = pd.DataFrame({
    'timestamp': dates,
    'open': close_prices + np.random.randn(n_candles) * 50,
    'high': close_prices + abs(np.random.randn(n_candles) * 100),
    'low': close_prices - abs(np.random.randn(n_candles) * 100),
    'close': close_prices,
    'volume': np.random.randint(100, 1000, n_candles),
})

# Validation qualité
assert len(df_market) > 1000, "❌ Pas assez de données"
assert df_market['close'].isna().sum() == 0, "❌ Valeurs manquantes détectées"
assert (df_market['high'] >= df_market['close']).all(), "❌ High < Close détecté"
assert (df_market['low'] <= df_market['close']).all(), "❌ Low > Close détecté"

print(f"✅ Données chargées: {len(df_market)} candles")
print(f"   Période: {df_market['timestamp'].min()} → {df_market['timestamp'].max()}")
print(f"   Prix range: ${df_market['close'].min():.0f} - ${df_market['close'].max():.0f}")
print(f"   Qualité: {(1 - df_market['close'].isna().sum() / len(df_market)) * 100:.1f}%")

In [ ]:
from itertools import product
import time

# Générer toutes les combinaisons de paramètres
param_names = list(SWEEP_CONFIG.keys())
param_values = list(SWEEP_CONFIG.values())
all_combinations = list(product(*param_values))

print(f"🔧 Génération de {len(all_combinations)} configurations...")

# Initialiser BacktestEngine (GPU si disponible)
strategy_class = get_strategy_class(STRATEGY_NAME)
engine = BacktestEngine(use_gpu=True)

# Exécuter backtests
results = []
start_time = time.time()

for i, combo in enumerate(all_combinations, 1):
    params = dict(zip(param_names, combo))
    
    try:
        # Backtest avec ces paramètres
        result = engine.run(
            strategy_class=strategy_class,
            data=df_market,
            params=params,
        )
        
        # Stocker résultats
        results.append({
            **params,
            "sharpe_ratio": result.sharpe_ratio,
            "total_return": result.total_return,
            "max_drawdown": result.max_drawdown,
            "win_rate": result.win_rate if hasattr(result, 'win_rate') else 0.0,
            "total_trades": result.total_trades if hasattr(result, 'total_trades') else 0,
        })
        
        if i % 5 == 0:
            print(f"   Progress: {i}/{len(all_combinations)} configs testées...")
            
    except Exception as e:
        print(f"   ⚠️ Config {i} failed: {e}")
        continue

elapsed = time.time() - start_time
df_sweep = pd.DataFrame(results)

print(f"\n✅ Sweep terminé en {elapsed:.1f}s")
print(f"   Configs valides: {len(df_sweep)}/{len(all_combinations)}")
print(f"   Vitesse: {len(df_sweep) / elapsed:.1f} tests/sec")
print(f"\n📊 Top 3 Sharpe:")
print(df_sweep.nlargest(3, 'sharpe_ratio')[['short_period', 'long_period', 'sharpe_ratio', 'max_drawdown']])

In [ ]:
# Initialiser l'agent Analyst
analyst = Analyst(model="deepseek-r1:70b", debug=True)

print("🔍 Lancement analyse Analyst (deepseek-r1:70b)...")
print("⏳ Cela peut prendre 30-60s selon le modèle...\n")

# Analyser les résultats du sweep
analysis_result = analyst.analyze_sweep_results(
    sweep_df=df_sweep,
    top_n=5
)

# Afficher résultats
print("=" * 80)
print("📊 RÉSULTATS ANALYSE ANALYST")
print("=" * 80)

print("\n🎯 PATTERNS IDENTIFIÉS:")
for i, pattern in enumerate(analysis_result['analysis']['patterns'], 1):
    print(f"   {i}. {pattern}")

print("\n📈 MÉTRIQUES CLÉS:")
for metric, value in analysis_result['analysis']['key_metrics'].items():
    print(f"   {metric}: {value}")

print("\n⚖️ TRADE-OFFS OBSERVÉS:")
for i, tradeoff in enumerate(analysis_result['analysis']['trade_offs'], 1):
    print(f"   {i}. {tradeoff}")

print("\n💡 RECOMMANDATIONS:")
for i, rec in enumerate(analysis_result['analysis']['recommendations'], 1):
    print(f"   {i}. {rec}")

print("\n✅ Analyse terminée")
print(f"   Métriques agent: {analyst.get_metrics()}")

In [ ]:
# Initialiser l'agent Strategist
strategist = Strategist(model="gpt-oss:20b", debug=True)

print("🎨 Lancement génération Strategist (gpt-oss:20b)...")
print("⏳ Génération de 3 propositions créatives...\n")

# Générer propositions basées sur l'analyse
proposals_result = strategist.propose_modifications(
    analysis=analysis_result,
    current_params=BASELINE_PARAMS,
    param_specs=PARAM_SPECS,
    n_proposals=3
)

# Afficher propositions
print("=" * 80)
print("💡 PROPOSITIONS STRATEGIST")
print("=" * 80)

for i, prop in enumerate(proposals_result['proposals'], 1):
    print(f"\n📌 PROPOSITION {i}: {prop['name']}")
    print(f"   Paramètres:")
    for param, value in prop['params'].items():
        baseline_val = BASELINE_PARAMS.get(param, "N/A")
        print(f"      {param}: {baseline_val} → {value}")
    print(f"   Rationale: {prop['rationale']}")

print(f"\n✅ Propositions générées: {proposals_result['total_valid']}/{proposals_result['total_generated']}")
print(f"   Métriques agent: {strategist.get_metrics()}")

In [ ]:
print("🚀 Tests automatiques des propositions LLM...\n")

# Tester baseline d'abord
print("📊 Test BASELINE...")
baseline_result = engine.run(
    strategy_class=strategy_class,
    data=df_market,
    params=BASELINE_PARAMS
)
print(f"   Sharpe: {baseline_result.sharpe_ratio:.3f}")
print(f"   Return: {baseline_result.total_return:.2%}")
print(f"   Drawdown: {baseline_result.max_drawdown:.2%}\n")

# Tester chaque proposition
proposal_results = []

for i, prop in enumerate(proposals_result['proposals'], 1):
    print(f"📊 Test PROPOSITION {i}: {prop['name']}...")
    
    try:
        result = engine.run(
            strategy_class=strategy_class,
            data=df_market,
            params=prop['params']
        )
        
        proposal_results.append({
            'name': prop['name'],
            'params': prop['params'],
            'sharpe_ratio': result.sharpe_ratio,
            'total_return': result.total_return,
            'max_drawdown': result.max_drawdown,
        })
        
        print(f"   Sharpe: {result.sharpe_ratio:.3f} ({result.sharpe_ratio - baseline_result.sharpe_ratio:+.3f})")
        print(f"   Return: {result.total_return:.2%} ({result.total_return - baseline_result.total_return:+.2%})")
        print(f"   Drawdown: {result.max_drawdown:.2%}\n")
        
    except Exception as e:
        print(f"   ❌ Erreur: {e}\n")
        continue

print("✅ Tests terminés")

In [ ]:
# Créer DataFrame comparatif
comparison_data = [{
    'Config': 'BASELINE',
    'Sharpe': baseline_result.sharpe_ratio,
    'Return': baseline_result.total_return,
    'Drawdown': baseline_result.max_drawdown,
}]

for res in proposal_results:
    comparison_data.append({
        'Config': res['name'],
        'Sharpe': res['sharpe_ratio'],
        'Return': res['total_return'],
        'Drawdown': res['max_drawdown'],
    })

df_comparison = pd.DataFrame(comparison_data)

# Visualisation 3 métriques
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Sharpe Ratio
axes[0].bar(df_comparison['Config'], df_comparison['Sharpe'], 
            color=['gray'] + ['steelblue']*len(proposal_results))
axes[0].axhline(baseline_result.sharpe_ratio, color='red', linestyle='--', label='Baseline')
axes[0].set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend()

# Total Return
axes[1].bar(df_comparison['Config'], df_comparison['Return'] * 100,
            color=['gray'] + ['green']*len(proposal_results))
axes[1].axhline(baseline_result.total_return * 100, color='red', linestyle='--', label='Baseline')
axes[1].set_title('Total Return Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Return (%)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

# Max Drawdown (inversé car plus bas = meilleur)
axes[2].bar(df_comparison['Config'], df_comparison['Drawdown'] * 100,
            color=['gray'] + ['coral']*len(proposal_results))
axes[2].axhline(baseline_result.max_drawdown * 100, color='red', linestyle='--', label='Baseline')
axes[2].set_title('Max Drawdown Comparison', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Drawdown (%)')
axes[2].tick_params(axis='x', rotation=45)
axes[2].legend()

plt.tight_layout()
plt.savefig('multi_llm_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualisations créées: multi_llm_comparison.png")

In [ ]:
# Identifier meilleure config
best_idx = df_comparison['Sharpe'].idxmax()
best_config = df_comparison.iloc[best_idx]

print("=" * 80)
print("🏆 RAPPORT FINAL - POC MULTI-LLM OPTIMIZER")
print("=" * 80)

print(f"\n📊 BASELINE:")
print(f"   Params: {BASELINE_PARAMS}")
print(f"   Sharpe: {baseline_result.sharpe_ratio:.3f}")
print(f"   Return: {baseline_result.total_return:.2%}")
print(f"   Drawdown: {baseline_result.max_drawdown:.2%}")

print(f"\n🏆 MEILLEURE CONFIG: {best_config['Config']}")
if best_idx > 0:  # Si pas baseline
    best_proposal = proposal_results[best_idx - 1]
    print(f"   Params: {best_proposal['params']}")
    print(f"   Sharpe: {best_config['Sharpe']:.3f} ({best_config['Sharpe'] - baseline_result.sharpe_ratio:+.3f})")
    print(f"   Return: {best_config['Return']:.2%} ({best_config['Return'] - baseline_result.total_return:+.2%})")
    print(f"   Drawdown: {best_config['Drawdown']:.2%}")
    
    improvement_pct = ((best_config['Sharpe'] - baseline_result.sharpe_ratio) / abs(baseline_result.sharpe_ratio)) * 100
    print(f"\n   💡 Amélioration Sharpe: {improvement_pct:+.1f}%")
else:
    print("   ⚠️ Baseline reste la meilleure config")

print(f"\n📈 RÉSUMÉ TABLEAU:")
print(df_comparison.to_string(index=False))

print(f"\n🤖 PERFORMANCE AGENTS:")
print(f"   Analyst (deepseek-r1:70b): {analyst.get_metrics()}")
print(f"   Strategist (gpt-oss:20b): {strategist.get_metrics()}")

print("\n✅ POC COMPLET TERMINÉ")
print("   Fichiers générés: multi_llm_comparison.png")
print("   Prochaines étapes: Itérer avec nouvelles propositions basées sur meilleure config")

## 📈 Section 8: Visualisation & Rapport Final

Comparaison visuelle des résultats et identification du meilleur config.

## ✅ Section 7: Validation Automatique (Tests GPU)

Exécution des backtests pour chaque proposition LLM.

## 🎨 Section 6: Propositions LLM-B (Strategist Agent)

Génération de 3 propositions créatives basées sur l'analyse.

## 🧠 Section 5: Analyse LLM-A (Analyst Agent)

Analyse quantitative des top 5 configs par l'agent Analyst.

## ⚙️ Section 4: Exécution du Sweep Initial (GPU)

Grid search sur 24 configurations de MA_Crossover.

## 📊 Section 3: Validation des données d'entrée

Chargement des données historiques et vérification de qualité.

In [ ]:
# Vérifier que Ollama est actif et modèles disponibles
try:
    llm_client = LLMClient(model="deepseek-r1:70b", timeout=60.0)
    test_response = llm_client.complete("Test: réponds juste 'OK'", max_tokens=10)
    print(f"✅ LLM Client OK: {test_response[:50]}")
except Exception as e:
    print(f"❌ ERREUR LLM: {e}")
    print("⚠️ Assure-toi que Ollama tourne: `ollama serve`")
    print("⚠️ Télécharge modèles: `ollama pull deepseek-r1:70b`")
    raise

## 📦 Section 1: Configuration de l'environnement

Import des modules ThreadX et configuration de base.